In [2]:
import pymysql
import pandas as pd
import numpy as np
from functools import reduce

# coding:utf_8

In [11]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [80]:
dict = pd.read_excel(r"C:\Users\13670\Desktop\1\dict副本.xlsx")

In [20]:
id_list = dict.loc[:,'id'].values.tolist()

tag_df = pd.DataFrame(columns=['ORDER_CODE', 'TAG_ID'])
qn_q_df = pd.DataFrame(columns=['ORDER_CODE', 'SD_CODE', 'abnormal'])

for items in id_list:
    id_str = "'"+ items + "'"
    if ('OBDIM')in id_str:
        tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM T66_APOLLO_ANALYSIS_TAG \
    WHERE SOURCE_TYPE = 2 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
        tag = pd.read_sql(tagsql,con=conn)
        tag_df = pd.concat([tag_df,tag],axis = 0)        
    elif ('N')in id_str:
        qnsql = 'SELECT ORDER_CODE,SD_CODE,\
    IF (IS_ABNORMAL = TRUE, 1,0) AS abnormal \
    FROM T66_ATHENA_BASIC_QUANTITY_RECORD \
    WHERE SZ_STATUS = 1 AND SD_CODE IN ('+id_str+')'
        qn = pd.read_sql(qnsql,con=conn)
        qn_q_df = pd.concat([qn_q_df,qn],axis = 0)
    else :
        qsql = 'SELECT ORDER_CODE,SD_CODE,\
    IF (IS_ABNORMAL = TRUE, 1,0) AS abnormal \
    FROM T66_ATHENA_BASIC_QUALITY_RECORD \
    WHERE SZ_STATUS = 1 AND SD_CODE IN ('+id_str+')'
        q = pd.read_sql(qsql,con=conn)
        qn_q_df = pd.concat([qn_q_df,q],axis = 0)
        
qn_q_duplicates = qn_q_df.drop_duplicates(subset=['ORDER_CODE', 'SD_CODE'])
final_qn_q_pivot = qn_q_duplicates.pivot(index='ORDER_CODE', columns='SD_CODE', values='abnormal')


final_tag_df_pivot = pd.DataFrame(columns=[])
tag_df["score"] = 1
tag_df_pivot = tag_df.pivot(index='ORDER_CODE',columns='TAG_ID',values='score')
for i in tag_df_pivot.loc[:,tag_df_pivot.columns.str.contains('EX')].columns:
    df = tag_df_pivot.loc[tag_df_pivot[i].notnull(),tag_df_pivot.columns.str.contains(i[:9])].fillna(0)
    final_tag_df_pivot = pd.merge(final_tag_df_pivot,df,left_index=True,right_index=True,how='outer')
final_tag_df_pivot    

,OBDIM0006EX,OBDIM0006Tag0010,OBDIM0006Tag0011,OBDIM0006Tag0012,OBDIM0006Tag0013,OBDIM0006Tag0014,OBDIM0006Tag0015,OBDIM0006Tag0016,OBDIM0006Tag0017,OBDIM0006Tag0018,...,OBDIM0006Tag0043,OBDIM0006Tag0045,OBDIM0006Tag0046,OBDIM0006Tag0047,OBDIM0006Tag0048,OBDIM0006Tag0049,OBDIM0006Tag0053,OBDIM0006Tag0054,OBDIM0006Tag0061,OBDIM0046EX
ORDER_CODE,,,,,,,,,,,,,,,,,,,,,
000000765491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
000003745904,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
000004527245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
000004598254,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000006786795,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999986953057,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
999988810200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
999990770784,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
person = pd.read_sql('SELECT ORDER_CODE,EXAMINEE_GENDER,EXAMINEE_AGE FROM T66_ATHENA_BASIC_RAW_PERSON_INFO WHERE SZ_STATUS =1',con=conn)
person.replace({'MW':"女",'UW':"女"},inplace=True)
person.replace('M',"男",inplace=True)
personp = person.set_index('ORDER_CODE')

KeyboardInterrupt: 

In [22]:
final_qn_q_pivot.to_pickle(r"C:\Users\13670\Desktop\1\肾囊肿定性定量.pkl")
final_tag_df_pivot.to_pickle(r"C:\Users\13670\Desktop\1\肾囊肿tag.pkl")  

In [3]:
qn_q = pd.read_pickle(r"C:\Users\13670\Desktop\1\肾囊肿定性定量.pkl")
tag = pd.read_pickle(r"C:\Users\13670\Desktop\1\肾囊肿tag.pkl")

In [10]:
qn_q

SD_CODE,S0000214N08,S0000215N79,S0000418N99,S0000419N33,S0000740N98,S0000740Q,S0000744N64,S0000744Q,S0000747N16,S0000747N96,S0000747Q,S1182254N30,S2104117N17,S2104117N49,S2104117Q,S3606753N66,S9610592N18,S9610592N74,S9610592Q
ORDER_CODE,,,,,,,,,,,,,,,,,,,
000000765491,NaN,0,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000001807184,0,NaN,0,0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000002165753,0,NaN,0,0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000003745904,0,0,1,0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004527245,0,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999990770784,0,0,0,0,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999990884325,0,NaN,0,0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999994241458,0,0,0,0,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tag

,OBDIM0006EX,OBDIM0006Tag0010,OBDIM0006Tag0011,OBDIM0006Tag0012,OBDIM0006Tag0013,OBDIM0006Tag0014,OBDIM0006Tag0015,OBDIM0006Tag0016,OBDIM0006Tag0017,OBDIM0006Tag0018,...,OBDIM0006Tag0043,OBDIM0006Tag0045,OBDIM0006Tag0046,OBDIM0006Tag0047,OBDIM0006Tag0048,OBDIM0006Tag0049,OBDIM0006Tag0053,OBDIM0006Tag0054,OBDIM0006Tag0061,OBDIM0046EX
ORDER_CODE,,,,,,,,,,,,,,,,,,,,,
000000765491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
000003745904,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
000004527245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
000004598254,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000006786795,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999986953057,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
999988810200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
999990770784,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
cols = [c for c in tag.columns if ("EX") not in c]
tag_new = tag[cols]

In [5]:
personp

NameError: name 'personp' is not defined

In [60]:
dig = tag.loc[:,["OBDIM0006Tag0028"]].dropna(axis = 0,how = "any")

In [91]:
p1 = pd.merge(dig,tag,on = "ORDER_CODE",how = "left" )
p2 = pd.merge(p1,qn_q,on = "ORDER_CODE",how = "left" )
p3 = pd.merge(p2,personp,on = "ORDER_CODE",how = "left" )
p_final = p3.drop(p3.loc[:,p3.columns.str.contains("EX")],axis =1)

In [178]:
dfs = [tag,qn_q,personp]

In [179]:
df_final = reduce(lambda left, right: pd.merge(left, right, on = "ORDER_CODE"), dfs)

In [180]:
df_final

,OBDIM0006EX,OBDIM0006Tag0010,OBDIM0006Tag0011,OBDIM0006Tag0012,OBDIM0006Tag0013,OBDIM0006Tag0014,OBDIM0006Tag0015,OBDIM0006Tag0016,OBDIM0006Tag0017,OBDIM0006Tag0018,...,S1182254N30,S2104117N17,S2104117N49,S2104117Q,S3606753N66,S9610592N18,S9610592N74,S9610592Q,EXAMINEE_GENDER,EXAMINEE_AGE
ORDER_CODE,,,,,,,,,,,,,,,,,,,,,
000000765491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,57
000003745904,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,55
000004527245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,33
000004598254,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,22
000006786795,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999986953057,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,51
999988810200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,37
999990770784,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,23


In [93]:
name = dict.set_index('id')['name'].to_dict()

In [94]:
p_(columns=name,inplace=True)

In [126]:
p_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391174 entries, 000003745904 to 999994398702
Data columns (total 58 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   OBDIM0006Tag0028_x  391174 non-null  float64
 1   肾混合回声               391174 non-null  float64
 2   肾积水                 391174 non-null  float64
 3   肾萎缩                 391174 non-null  float64
 4   肾缩小                 391174 non-null  float64
 5   肾萎缩                 391174 non-null  float64
 6   肾缩小                 391174 non-null  float64
 7   肾萎缩                 391174 non-null  float64
 8   肾缩小                 391174 non-null  float64
 9   肾增大                 391174 non-null  float64
 10  肾增大                 391174 non-null  float64
 11  肾增大                 391174 non-null  float64
 12  肾回声不匀               391174 non-null  float64
 13  肾缺失                 391174 non-null  float64
 14  肾缺失                 391174 non-null  float64
 15  肾结石                 39

In [99]:
P_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391174 entries, 000003745904 to 999994398702
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   OBDIM0006Tag0028_x  391174 non-null  float64
 1   肾混合回声               391174 non-null  float64
 2   肾积水                 391174 non-null  float64
 3   肾萎缩                 391174 non-null  float64
 4   肾缩小                 391174 non-null  float64
 5   肾增大                 391174 non-null  float64
 6   肾回声不匀               391174 non-null  float64
 7   肾缺失                 391174 non-null  float64
 8   肾结石                 391174 non-null  float64
 9   OBDIM0006Tag0028_y  391174 non-null  float64
 10  肾错构瘤                391174 non-null  float64
 11  肾异常回声               391174 non-null  float64
 12  肾占位                 391174 non-null  float64
 13  肾发育异常               391174 non-null  float64
 14  肾钙化灶                391174 non-null  float64
 15  肾回声增粗               39

In [101]:
dt2= dt.map(lambda x: re.sub('sichuan','SC',x))

ValueError: level > 0 or level < -1 only valid with MultiIndex

In [127]:
P_final = p_final.sum(axis = 1,level = 0,skipna = True)

In [143]:
P_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391174 entries, 000003745904 to 999994398702
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   OBDIM0006Tag0028_x  391174 non-null  float64
 1   肾混合回声               391174 non-null  float64
 2   肾积水                 391174 non-null  float64
 3   肾萎缩                 391174 non-null  float64
 4   肾缩小                 391174 non-null  float64
 5   肾增大                 391174 non-null  float64
 6   肾回声不匀               391174 non-null  float64
 7   肾缺失                 391174 non-null  float64
 8   肾结石                 391174 non-null  float64
 9   OBDIM0006Tag0028_y  391174 non-null  float64
 10  肾错构瘤                391174 non-null  float64
 11  肾异常回声               391174 non-null  float64
 12  肾占位                 391174 non-null  float64
 13  肾发育异常               391174 non-null  float64
 14  肾钙化灶                391174 non-null  float64
 15  肾回声增粗               39

In [ ]:
if df.columns.duplicated().any():
    all_levels = df.columns.nlevels
    if all_levels > 1:
        all_levels = range(all_levels)
    df = df.groupby(axis=1, level=all_levels).sum()

In [117]:
data = {"id1":[0,0,0],"id1":[1,1,1],"id2":[2,2,2],"id2":[3,3,3]}
test_df = pd.DataFrame(data,columns = ["id1","id1","id2","id2",])

In [116]:
test_df

,id1,id1,id2,id2
0,1,1,3,3
1,1,1,3,3
2,1,1,3,3


In [122]:
Test_df =test_df.sum(axis = 1,level = 0,skipna = False)

In [123]:
Test_df

,id1,id2
0,2,6
1,2,6
2,2,6


In [21]:
test_df.loc[:,["id","name"]]

,id,name
0,1,A
1,2,B
2,3,C


In [37]:
test_df.loc[:,test_df.columns.str.contains("n")]

,name
0,A
1,B
2,C


In [5]:
ceshi = pd.read_excel(r"C:\Users\13670\Desktop\ceshi.xlsx")
ceshi

,id1,id1.1,id2,id2.1,id3,id3.1
0,0,1,2,NaN,NaN,4
1,0,1,2,3.0,NaN,4
2,0,1,2,NaN,NaN,4
3,0,1,2,3.0,NaN,4


In [3]:
ceshi1 = ceshi.sum(axis = 1,level = 0,skipna = False)
ceshi1

NameError: name 'ceshi' is not defined

In [12]:
ceshi1.applymap(lambda x:1 if x >=1 and notnull(x), x:0 elif x =0, else NA)

SyntaxError: invalid syntax (<ipython-input-12-3c048aca32df>, line 1)

In [198]:
ceshi1["id2.1"].apply(lambda x:1 if x >=1 else 0)

0    0
1    1
2    0
3    1
Name: id2.1, dtype: int64

In [196]:
for i in ceshi1.columns.values.tolist():
    ceshi["i"]
#     ceshi1["i"].map(lambda x:1 if x >=1 else 0)
# ceshi1

KeyError: 'i'

In [182]:
ceshi2 = ceshi1["i"].map(lambda x:1 if x >=1 else 0)

AttributeError: 'DataFrame' object has no attribute 'map'